In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!cp "/content/drive/MyDrive/archive.zip" /content/

In [3]:
!unzip -q "/content/drive/MyDrive/archive.zip" -d /content/

In [5]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.6/890.6 kB 14.2 MB/s eta 0:00:00


In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim
from PIL import Image
import os
import numpy as np

In [17]:
# Define custom dataset to load raw and reference images
class ImageEnhancementDataset(Dataset):
    def __init__(self, raw_dir, ref_dir, transform=None):
        self.raw_paths = [os.path.join(raw_dir, f) for f in os.listdir(raw_dir)]
        self.ref_paths = [os.path.join(ref_dir, f) for f in os.listdir(ref_dir)]
        self.transform = transform

    def __len__(self):
        return len(self.raw_paths)

    def __getitem__(self, idx):
        raw_img = Image.open(self.raw_paths[idx]).convert("RGB")
        ref_img = Image.open(self.ref_paths[idx]).convert("RGB")

        if self.transform:
            raw_img = self.transform(raw_img)
            ref_img = self.transform(ref_img)

        return raw_img, ref_img


In [21]:
# Define forward diffusion process with linear schedule
def forward_diffusion(x0, noise, t, T):
    t = t.view(-1, 1, 1, 1)
    alpha = 1 - (t / T)
    return alpha * x0 + (1 - alpha) * noise

# U-Net model adapted for 224x224 input images
class SimpleUNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(SimpleUNet, self).__init__()

        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU()
        )

        self.decoder = nn.Sequential(
            nn.Conv2d(128, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, out_channels, kernel_size=3, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


In [22]:
# Hyperparameters
T = 1000
batch_size = 16
learning_rate = 1e-4
epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])

# Load dataset
train_dataset = ImageEnhancementDataset("/content/Train/Raw", "/content/Train/Reference", transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Model, optimizer, loss
model = SimpleUNet(in_channels=3, out_channels=3).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

# Training loop
for epoch in range(epochs):
    model.train()
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        # Forward diffusion
        noise = torch.randn_like(data)
        t = torch.randint(0, T, (data.shape[0],)).to(device)
        xt = forward_diffusion(data, noise, t, T)

        # Denoise step
        reconstructed = model(xt)
        loss = criterion(reconstructed, target)

        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}')


Epoch 1/10, Loss: 0.05486708879470825
Epoch 2/10, Loss: 0.03904678300023079
Epoch 3/10, Loss: 0.03909691423177719
Epoch 4/10, Loss: 0.03408442810177803
Epoch 5/10, Loss: 0.027514753863215446
Epoch 6/10, Loss: 0.0422213040292263
Epoch 7/10, Loss: 0.04622916877269745
Epoch 8/10, Loss: 0.0428997278213501
Epoch 9/10, Loss: 0.039231400936841965
Epoch 10/10, Loss: 0.03697602078318596


In [27]:
from skimage.metrics import structural_similarity as ssim

# Testing loop with MAE, SSIM, and PSNR
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)

        # Diffuse and denoise
        noise = torch.randn_like(data)
        t = torch.tensor([T-1], device=device).expand(data.shape[0])
        xt = forward_diffusion(data, noise, t, T)
        reconstructed = model(xt)

        # Move to CPU for metric calculation
        reconstructed_np = reconstructed.cpu().numpy().transpose(0, 2, 3, 1)
        target_np = target.cpu().numpy().transpose(0, 2, 3, 1)

        for rec, tgt in zip(reconstructed_np, target_np):
            mae_sum += np.mean(np.abs(rec - tgt))
            ssim_sum += ssim(rec, tgt, channel_axis=-1, data_range=1.0)
            psnr_sum += psnr(tgt, rec, data_range=1.0)

        num_batches += data.size(0)

# Compute average metrics
mae_avg = mae_sum / num_batches
ssim_avg = ssim_sum / num_batches
psnr_avg = psnr_sum / num_batches

print(f"Test MAE: {mae_avg:.4f}, SSIM: {ssim_avg:.4f}, PSNR: {psnr_avg:.4f}")


Test MAE: 0.2130, SSIM: 0.1199, PSNR: 12.1094
